In [ ]:
#Prompt: Explain what SQL is and demonstrate CREATE TABLE, PRIMARY KEY, and INSERT statements with code, explaining why each is required to store structured data.

### What is SQL?

SQL stands for **Structured Query Language**. It is used to communicate with a database. While there are many different flavors of SQL (like PostgreSQL, MySQL, and SQLite), they all share a common core syntax for defining and manipulating data.

To store structured data, we need to define **what** we are storing and ensure that each piece of data is unique and reachable. That's where `CREATE TABLE`, `PRIMARY KEY`, and `INSERT` come in.

In [ ]:
import sqlite3

# 1. Connect to an in-memory database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# 2. CREATE TABLE with a PRIMARY KEY
# This defines the schema (structure) of our data.
cursor.execute('''
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    major TEXT
)
''')

# 3. INSERT statements
# This adds actual records into our structured table.
cursor.execute("INSERT INTO students (name, major) VALUES ('Alice', 'Computer Science')")
cursor.execute("INSERT INTO students (name, major) VALUES ('Bob', 'Mathematics')")

# Commit changes and display the results
connection.commit()

cursor.execute("SELECT * FROM students")
results = cursor.fetchall()

print("Contents of 'students' table:")
for row in results:
    print(row)

connection.close()

### Why these statements are required:

1.  **`CREATE TABLE`**: This defines the **Schema**. In structured data, you must define the types of data (strings, integers, etc.) and the column names before you can store anything. It acts as the blueprint for your data.
2.  **`PRIMARY KEY`**: This is a constraint that uniquely identifies each record in a table.
    *   **Uniqueness**: It ensures no two rows are identical (e.g., two students named 'Alice' can be distinguished by their ID).
    *   **Integrity**: It prevents null values in that column, ensuring every record has an identity.
3.  **`INSERT`**: This is the Data Manipulation Language (DML) command used to populate the table. Without `INSERT`, your database structure would remain empty.

In [ ]:
#Prompt: Explain SELECT, WHERE, DISTINCT, ORDER BY, and LIMIT/TOP using SQL code and why these clauses are essential for retrieving meaningful and sorted data.

### Data Retrieval in SQL

Once data is stored, we use retrieval clauses to find exactly what we need. These clauses transform a raw table into meaningful information.

*   **`SELECT`**: Chooses specific columns.
*   **`WHERE`**: Filters rows based on a condition.
*   **`DISTINCT`**: Removes duplicate values from the output.
*   **`ORDER BY`**: Sorts the results.
*   **`LIMIT`**: Restricts the number of rows returned.

In [ ]:
import sqlite3

# Setup: Create a fresh database and add more diverse data
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

cursor.execute('''
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    major TEXT,
    gpa REAL
)
''')

# Inserting multiple records to demonstrate filtering and sorting
student_data = [
    ('Alice', 'Computer Science', 3.8),
    ('Bob', 'Mathematics', 3.5),
    ('Charlie', 'Computer Science', 3.9),
    ('Diana', 'Physics', 3.7),
    ('Edward', 'Mathematics', 3.2),
    ('Alice', 'Biology', 3.6) # Duplicate name for DISTINCT demo
]
cursor.executemany("INSERT INTO students (name, major, gpa) VALUES (?, ?, ?)", student_data)

def run_query(title, query):
    print(f"--- {title} ---")
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)
    print("\n")

# 1. SELECT - Choosing specific columns
run_query("SELECT: Just names and GPAs", "SELECT name, gpa FROM students")

# 2. WHERE - Filtering by condition
run_query("WHERE: Only Computer Science students", "SELECT * FROM students WHERE major = 'Computer Science'")

# 3. DISTINCT - Unique values only
run_query("DISTINCT: Unique majors available", "SELECT DISTINCT major FROM students")

# 4. ORDER BY - Sorting by GPA (Descending)
run_query("ORDER BY: Students by GPA (High to Low)", "SELECT * FROM students ORDER BY gpa DESC")

# 5. LIMIT - Restricting result count
run_query("LIMIT: Top 3 students", "SELECT * FROM students ORDER BY gpa DESC LIMIT 3")

connection.close()

### Why these clauses are essential:

1.  **`SELECT` (Focus)**: Databases can have hundreds of columns. Selecting only what you need reduces memory usage and makes the output readable.
2.  **`WHERE` (Precision)**: In real-world databases with millions of rows, you rarely want everything. `WHERE` allows you to find specific records (like a specific user or a specific date range).
3.  **`DISTINCT` (Cleanup)**: Useful for finding categories. For example, finding all unique cities where customers live without seeing the same city repeated thousands of times.
4.  **`ORDER BY` (Organization)**: Data is often stored in an unpredictable order. Sorting is vital for reports, leaderboards, or chronological logs.
5.  **`LIMIT` (Efficiency)**: Prevents the system from being overwhelmed by too much data. It's used for "Top 10" lists or for "Pagination" (showing 20 results per page).

In [ ]:
#Prompt: Explain UPDATE and DELETE with SQL examples and why modifying or removing records must be done carefully.

### Modifying and Deleting Data

Sometimes data changes (a student changes their major) or needs to be removed (a student graduates). SQL provides two primary commands for this:

*   **`UPDATE`**: Modifies existing records in a table.
*   **`DELETE`**: Removes existing records from a table.

In [ ]:
import sqlite3

# Setup: Fresh database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

cursor.execute('''
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    major TEXT
)
''')

student_data = [
    ('Alice', 'Computer Science'),
    ('Bob', 'Mathematics'),
    ('Charlie', 'Physics')
]
cursor.executemany("INSERT INTO students (name, major) VALUES (?, ?)", student_data)

def show_table(msg):
    print(f"--- {msg} ---")
    cursor.execute("SELECT * FROM students")
    for row in cursor.fetchall():
        print(row)
    print("\n")

show_table("Initial Data")

# 1. UPDATE - Changing Alice's major to Data Science
# Crucial: Without the WHERE clause, EVERY student's major would change!
cursor.execute("UPDATE students SET major = 'Data Science' WHERE name = 'Alice'")
show_table("After UPDATE (Alice switched to Data Science)")

# 2. DELETE - Removing Bob from the table
# Crucial: Without the WHERE clause, EVERY row would be deleted!
cursor.execute("DELETE FROM students WHERE name = 'Bob'")
show_table("After DELETE (Bob removed)")

connection.close()

### Why Caution is Required:

1.  **The "Point of No Return"**: Unlike a document where you can hit "Undo," once a `DELETE` or `UPDATE` command is committed in many database environments, the data is changed permanently unless you have backups or transaction logs.
2.  **The Missing `WHERE` Clause**: This is one of the most common and dangerous mistakes in SQL.
    *   `UPDATE students SET major = 'History'` (No WHERE clause) -> **Everyone** is now a History major.
    *   `DELETE FROM students` (No WHERE clause) -> The table is now **empty**.
3.  **Data Integrity**: Deleting a record might break relationships. If you delete a student, but their "Grades" still exist in another table, you end up with "orphaned" data that doesn't belong to anyone.

**Best Practice**: Always run a `SELECT` statement with your `WHERE` clause first to verify exactly which rows will be affected before you run the `UPDATE` or `DELETE`.

In [ ]:
#Prompt: Explain aggregate functions (COUNT, SUM, AVG, MIN, MAX) along with GROUP BY and HAVING, showing SQL code and why they are used for data analysis.

### Aggregation and Grouping in SQL

Aggregate functions perform a calculation on a set of values and return a single value. They are almost always used with the `GROUP BY` clause to organize results into summary categories.

*   **`COUNT()`**: Returns the number of rows.
*   **`SUM()`**: Returns the total sum of a numeric column.
*   **`AVG()`**: Returns the average value of a numeric column.
*   **`MIN()` / `MAX()`**: Returns the smallest and largest values.
*   **`GROUP BY`**: Groups rows that have the same values into summary rows.
*   **`HAVING`**: Similar to `WHERE`, but used to filter groups created by `GROUP BY`.

In [ ]:
import sqlite3

# Setup: Fresh database with sales data
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

cursor.execute('''
CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY,
    product_category TEXT,
    amount REAL,
    region TEXT
)
''')

sales_data = [
    ('Electronics', 1200.0, 'North'),
    ('Electronics', 800.0, 'South'),
    ('Furniture', 300.0, 'North'),
    ('Furniture', 450.0, 'North'),
    ('Clothing', 50.0, 'South'),
    ('Clothing', 150.0, 'North'),
    ('Electronics', 1100.0, 'North')
]
cursor.executemany("INSERT INTO sales (product_category, amount, region) VALUES (?, ?, ?)", sales_data)

def run_query(title, query):
    print(f"--- {title} ---")
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)
    print("\n")

# 1. Basic Aggregation
run_query("Basic Aggregation: Total Sales and Average Amount",
          "SELECT COUNT(*), SUM(amount), AVG(amount) FROM sales")

# 2. GROUP BY - Sales per Category
run_query("GROUP BY: Total Sales per Category",
          "SELECT product_category, SUM(amount) FROM sales GROUP BY product_category")

# 3. GROUP BY with multiple columns
run_query("GROUP BY: Category and Region",
          "SELECT product_category, region, COUNT(*) FROM sales GROUP BY product_category, region")

# 4. HAVING - Filtering groups
# Find categories where the total sales amount is greater than 500
run_query("HAVING: Categories with > 500 in total sales",
          "SELECT product_category, SUM(amount) FROM sales GROUP BY product_category HAVING SUM(amount) > 500")

connection.close()

### Why these are essential for Data Analysis:

1.  **Summarization**: Raw data is often too granular. Aggregates allow you to see the "big picture" (e.g., total monthly revenue instead of individual transactions).
2.  **Comparison**: `GROUP BY` allows you to compare different segments of your business or data (e.g., Which region is performing better? Which product category is the most popular?).
3.  **Insight Filtering (`HAVING`)**: While `WHERE` filters individual records *before* they are grouped, `HAVING` filters the results *after* aggregation. This is how you answer questions like "Which departments exceeded their budget?" or "Which customers have made more than 10 purchases?"
4.  **Trend Spotting**: Using `MIN` and `MAX` helps identify outliers or range boundaries in your dataset, such as the highest and lowest prices ever paid for a product.

In [ ]:
#Prompt: Explain all types of SQL joins — INNER JOIN, LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN, CROSS JOIN, and SELF JOIN — using clear SQL code examples on sample tables, and explain why each join is used, what result it produces, and when it should be applied in real-world scenarios.

### The SQL Join Family

Joins are used to combine rows from two or more tables based on a related column between them. The primary difference between join types is how they handle data that *doesn't* have a match in the other table.

In [ ]:
import sqlite3
import pandas as pd

# Setup: Fresh database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# Create tables
cursor.execute("CREATE TABLE customers (id INTEGER, name TEXT)")
cursor.execute("CREATE TABLE orders (order_id INTEGER, cust_id INTEGER, amount REAL)")

# Insert sample data
# Alice (1) and Bob (2) have orders. Charlie (3) has NO orders.
cursor.executemany("INSERT INTO customers VALUES (?, ?)", [(1, 'Alice'), (2, 'Bob'), (3, 'Charlie')])

# Order 103 refers to customer 5 (who doesn't exist in our table)
cursor.executemany("INSERT INTO orders VALUES (?, ?, ?)", [
    (101, 1, 50.0),
    (102, 2, 75.0),
    (103, 5, 20.0)
])

def show_query(title, sql):
    print(f"--- {title} ---")
    df = pd.read_sql_query(sql, connection)
    display(df)
    print("\n")

# 1. INNER JOIN: Returns only rows with matches in BOTH tables.
# Use case: Find customers who have actually placed an order.
show_query("INNER JOIN (Only Matches)",
           "SELECT name, order_id, amount FROM customers INNER JOIN orders ON customers.id = orders.cust_id")

# 2. LEFT JOIN: All from Left table, plus matches from Right (NULLs if no match).
# Use case: List ALL customers and their order status (including those with zero orders).
show_query("LEFT JOIN (All Customers)",
           "SELECT name, order_id, amount FROM customers LEFT JOIN orders ON customers.id = orders.cust_id")

# 3. RIGHT JOIN: All from Right table, plus matches from Left (NULLs if no match).
# Use case: Find all orders, including 'orphaned' orders that aren't linked to a valid customer.
show_query("RIGHT JOIN (All Orders)",
           "SELECT name, order_id, amount FROM customers RIGHT JOIN orders ON customers.id = orders.cust_id")

# 4. FULL OUTER JOIN: All rows from both tables, filling NULLs where there is no match.
# Use case: Complete audit of the relationship - find missing customers AND missing orders.
show_query("FULL OUTER JOIN (Everything)",
           "SELECT name, order_id, amount FROM customers FULL OUTER JOIN orders ON customers.id = orders.cust_id")

# 5. CROSS JOIN: Cartesian Product (Every combination of row A with row B).
# Use case: Generating combinations, like all possible T-shirt colors and sizes.
show_query("CROSS JOIN (Combinations)",
           "SELECT customers.name, orders.order_id FROM customers CROSS JOIN orders LIMIT 5")

# 6. SELF JOIN: Joining a table to itself.
# Use case: Hierarchical data, like an Employees table where one column is 'manager_id' referring to another 'emp_id'.
cursor.execute("CREATE TABLE staff (id INTEGER, name TEXT, manager_id INTEGER)")
cursor.executemany("INSERT INTO staff VALUES (?, ?, ?)", [(1, 'Boss', NULL), (2, 'Alice', 1), (3, 'Bob', 1)])
show_query("SELF JOIN (Manager/Subordinate)",
           "SELECT e.name AS Employee, m.name AS Manager FROM staff e JOIN staff m ON e.manager_id = m.id")

connection.close()

### When to use each Join:

| Join Type | Result | Real-World Scenario |
| :--- | :--- | :--- |
| **INNER** | Only the "Intersection". | Generating an invoice for active sales. |
| **LEFT** | Left table is preserved. | Creating a report of all registered users, even those who haven't posted yet. |
| **RIGHT** | Right table is preserved. | Checking for data errors (e.g., finding sales records that lost their customer link). |
| **FULL** | Both tables preserved. | Merging two different mailing lists into one master list. |
| **CROSS** | Multiplication of rows. | Creating a schedule for every employee on every possible shift day. |
| **SELF** | Logic within one table. | Finding pairs (e.g., finding all employees who work in the same city as their manager). |

In [ ]:
#Prompt: Explain INDEX, VIEW, and TRANSACTION (BEGIN, COMMIT, ROLLBACK) with SQL examples and why they are important for performance, security, and data consistency.

### Advanced SQL: Indexes, Views, and Transactions

Beyond basic data manipulation, SQL provides tools to optimize performance, simplify data access, and ensure reliability.

*   **`INDEX`**: A pointer to data in a table. It works like the index in a book, allowing the database to find rows without scanning the entire table.
*   **`VIEW`**: A virtual table based on the result-set of an SQL statement. It doesn't store data itself but provides a specific way to look at existing data.
*   **`TRANSACTION`**: A sequence of one or more SQL operations that are treated as a single unit of work. They follow the **ACID** properties (Atomicity, Consistency, Isolation, Durability).

In [ ]:
import sqlite3

# Setup: Fresh database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

cursor.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, email TEXT, balance REAL)')
cursor.executemany('INSERT INTO users (name, email, balance) VALUES (?, ?, ?)', [
    ('Alice', 'alice@example.com', 100.0),
    ('Bob', 'bob@example.com', 50.0)
])

# 1. INDEX - Speeding up lookups on the email column
cursor.execute("CREATE INDEX idx_user_email ON users(email)")
print("--- Index 'idx_user_email' created ---\n")

# 2. VIEW - Creating a 'secure' view that hides emails and IDs
cursor.execute("CREATE VIEW public_user_balances AS SELECT name, balance FROM users")

print("--- Querying the VIEW ---")
cursor.execute("SELECT * FROM public_user_balances")
for row in cursor.fetchall():
    print(row)
print("\n")

# 3. TRANSACTION - Moving money between accounts safely
try:
    # Start the transaction
    connection.execute("BEGIN TRANSACTION")

    # Step A: Subtract from Alice
    cursor.execute("UPDATE users SET balance = balance - 30 WHERE name = 'Alice'")

    # Simulate a potential error (e.g., system crash or logic check failure)
    # If we decided to stop here, we would ROLLBACK.

    # Step B: Add to Bob
    cursor.execute("UPDATE users SET balance = balance + 30 WHERE name = 'Bob'")

    # If everything succeeded, commit the changes
    connection.commit()
    print("--- Transaction Committed Successfully ---")
except Exception as e:
    # If any error occurred, undo everything in the transaction
    connection.rollback()
    print(f"--- Transaction Failed and Rolled Back: {e} ---")

# Show final balances
cursor.execute("SELECT name, balance FROM users")
print("\nFinal Balances:", cursor.fetchall())

connection.close()

### Why these are critical:

1.  **`INDEX` (Performance)**: Without an index, a database must perform a "Full Table Scan." For a table with 10 million rows, finding one email address would take seconds; with an index, it takes milliseconds.
2.  **`VIEW` (Security & Simplicity)**:
    *   **Security**: You can give a user access to a `VIEW` that contains only non-sensitive columns while denying access to the raw table containing passwords or personal info.
    *   **Simplicity**: If you have a massive join query that you run every day, you can save it as a View and simply query `SELECT * FROM my_view` instead of re-writing the complex logic.
3.  **`TRANSACTION` (Consistency)**:
    *   **The All-or-Nothing Rule**: Transactions prevent partial updates. In the bank transfer example, it's unacceptable for the money to be taken from Alice but never arrive for Bob. Transactions ensure that if one step fails, the entire operation is undone (`ROLLBACK`).
    *   **Concurrency**: They prevent different users from making conflicting changes to the same piece of data at the exact same time.